In [ ]:
using JuMP
using Ipopt
using Statistics

In [ ]:
function ejMarkowitz()
    stock_data = [
    93.043 51.826 1.063;
    84.585 52.823 0.938;
    111.453 56.477 1.000;
    99.525 49.805 0.938;
    95.819 50.287 1.438;
    114.708 51.521 1.700;
    111.515 51.531 2.540;
    113.211 48.664 2.390;
    104.942 55.744 3.120;
    99.827 47.916 2.980;
    91.607 49.438 1.900;
    107.937 51.336 1.750;
    115.590 55.081 1.800;
    ]
    t, n = size(stock_data)

    stock_returns = zeros(t-1, n)
    for i in 1:t-1
        stock_returns[i, :] = (stock_data[i + 1, :] .- stock_data[i, :]) ./ stock_data[i, :]
    end

    r = Statistics.mean(stock_returns, dims=1)
    Q = Statistics.cov(stock_returns, dims=1)
    
    portfolio = JuMP.Model(Ipopt.Optimizer)
    
    @variable(portfolio, x[1:n] >= 0)

    @objective(portfolio, Min, x' * Q * x)
    
    @constraint(portfolio, sum(x) <= 1_000)
    @constraint(portfolio, sum(r .* x) >= 50)
    
    println(portfolio)
    
    JuMP.set_optimizer_attribute(portfolio, "print_level", 0)
    JuMP.optimize!(portfolio)
    
    println("z = ", JuMP.objective_value(portfolio), " status = ", JuMP.primal_status(portfolio))
    Rpval = JuMP.value.(sum(x .* r))
    Vpval = JuMP.value.(x' * Q * x)
    DSpval = sqrt(Vpval)
    println("Rp = ", Rpval)
    println("DSp = ", DSpval)
    println("Vp = ", Vpval)
    xval = JuMP.value.(x)
    xval = round.(xval, digits=4)
    println("xval = ", xval)
    
    return
end

In [ ]:
ejMarkowitz()